In [1]:
import sys

import numpy as np
from scipy import integrate
from struct import pack, unpack

sys.path.insert(0, "../")
from paths import FIG_DIR, DATA_DIR

# Numerical integration

In [2]:
def compute_lcs_volume(p, ssb):
    normal = ssb @ p
    if len(np.unique(normal)) < len(normal):
        return 1  # upper bound
    vertices_lcs = list(np.where(normal < 0)[0])
    normal_short = normal[vertices_lcs]
    with np.errstate(divide='ignore'):
        factors = normal_short[:, None]/(normal_short[:, None] - normal[None, :])
        factors = np.nan_to_num(factors, copy=False, nan=1.0, posinf=1.0, neginf=1.0)
    summands = np.prod(factors, axis=1)
    return np.sum(summands)

In [3]:
def f4(z, y, x, ssb_matrix):
    p = np.array([x, y, z, 1-x-y-z])
    return compute_lcs_volume(p, ssb_matrix)**2

m = 4  #number of alternative
# transitive preferences
ssb_tr = np.triu(np.ones((m, m))) - np.tril(np.ones((m, m)))
# Condorcet winner and 3 cycle
ssb_intr1 = np.array([[0, 1, 1, 1], [-1, 0, 1, -1], [-1, -1, 0, 1], [-1, 1, -1, 0]])
# 4 cycle
ssb_intr2 = np.array([[0, 1, 1, -1], [-1, 0, 1, 1], [-1, -1, 0, 1], [1, -1, -1, 0]])


val, err = integrate.tplquad(f4, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_tr,))
print(6*val, 6*err, 1-18*val, 18*err)
val, err = integrate.tplquad(f4, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_intr1,))
print(6*val, 6*err, 1-18*val, 18*err)
val, err = integrate.tplquad(f4, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_intr2,))
print(6*val, 6*err, 1-18*val, 18*err)

0.3328977573892963 8.901932002162174e-08 0.0013067278321111653 2.670579600648652e-07
0.31733609039911825 8.939711291365652e-08 0.04799172880264524 2.6819133874096955e-07
0.2928792895320537 8.938956645260067e-08 0.12136213140383878 2.68168699357802e-07


# Monte-Carlo integration

In [2]:
def compute_lcs_volume_vectorized(p_matr, ssb):
    normal = - p_matr @ ssb
    normal_short = np.where(normal < 0, normal, 0)
    with np.errstate(divide='ignore'):
        factors = normal_short[:, :, None] / (normal_short[:, :, None] - normal[:, None, :])
        factors = np.nan_to_num(factors, copy=False, nan=1.0, posinf=1.0, neginf=1.0)
    summands = np.prod(factors, axis=2)
    return np.sum(summands, axis=1)

In [7]:
num_samples = 7.95 * 10**10
m = 4
rng = np.random.default_rng(2025)
ssb_tr = np.triu(np.ones((m, m))) - np.tril(np.ones((m, m)))
alpha = np.ones(m)
batch_size = 10**7 #2.5sec for 10**7 samples

continued = True
if continued:
    with open(DATA_DIR / '5_2_MC_transitive.dat', 'rb') as file:
        packed = file.read()
        batch_results = list(unpack('d' * (len(packed) // 8), packed)) # 8 bytes per double
        print(len(batch_results))
else:
    batch_results = []

for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
    p_matr = rng.dirichlet(alpha, batch_size)
    samples = compute_lcs_volume_vectorized(p_matr, ssb_tr)
    batch_results.append((samples**2).mean())

with open(DATA_DIR / '5_2_MC_transitive.dat', 'wb') as file:
    file.write(pack('d' * len(batch_results) , *batch_results))

print(np.mean(batch_results), np.std(batch_results))

500
0.3329002673995511 9.412509687202058e-05


In [9]:
np.mean(batch_results), len(batch_results), 1-3* np.mean(batch_results)

(0.3329002673995511, 7950, 0.0012991978013467964)

In [3]:
num_samples = 7.95 * 10**10
m = 4
rng = np.random.default_rng(2026)
# 3 cycle not preferred to the last option
ssb_intr1 = np.array([[0, 1, 1, 1], [-1, 0, 1, -1], [-1, -1, 0, 1], [-1, 1, -1, 0]])
alpha = np.ones(m)
batch_size = 10**7 #2.5sec for 10**7 samples

continued = False
if continued:
    with open(DATA_DIR / '5_2_MC_intr1.dat', 'rb') as file:
        packed = file.read()
        batch_results = list(unpack('d' * (len(packed) // 8), packed)) # 8 bytes per double
else:
    batch_results = []

for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
    p_matr = rng.dirichlet(alpha, batch_size)
    samples = compute_lcs_volume_vectorized(p_matr, ssb_intr1)
    batch_results.append((samples**2).mean())

with open(DATA_DIR / '5_2_MC_intr1.dat', 'wb') as file:
    file.write(pack('d' * len(batch_results) , *batch_results))

print(np.mean(batch_results), np.std(batch_results)), len(batch_results), 1-3* np.mean(batch_results)

0.3173383865156588 9.104764179802117e-05


(None, 7950, 0.047984840453023536)

In [6]:
print(np.mean(batch_results), np.std(batch_results), len(batch_results), 1-3* np.mean(batch_results))

0.3173383865156588 9.104764179802117e-05 7950 0.047984840453023536


In [3]:
num_samples = 7.95 * 10**10
m = 4
rng = np.random.default_rng(2027)
# 4 cycle
ssb_intr2 = np.array([[0, 1, 1, -1], [-1, 0, 1, 1], [-1, -1, 0, 1], [1, -1, -1, 0]])
alpha = np.ones(m)
batch_size = 10**7 #2.5sec for 10**7 samples

continued = False
if continued:
    with open(DATA_DIR / '5_2_MC_intr2.dat', 'rb') as file:
        packed = file.read()
        batch_results = list(unpack('d' * (len(packed) // 8), packed)) # 8 bytes per double
else:
    batch_results = []

for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
    p_matr = rng.dirichlet(alpha, batch_size)
    samples = compute_lcs_volume_vectorized(p_matr, ssb_intr2)
    batch_results.append((samples**2).mean())

with open(DATA_DIR / '5_2_MC_intr2.dat', 'wb') as file:
    file.write(pack('d' * len(batch_results) , *batch_results))

print(np.mean(batch_results), np.std(batch_results), len(batch_results), 1-3*np.mean(batch_results))

0.29287948245662515 6.639823249458161e-05 7950 0.1213615526301246


# Simulation

In [2]:
num_samples = 2.65 * 10**10 #1 sample corresponds to 3 lotteries, a triple!
m = 4
rng = np.random.default_rng(2027)
ssb_tr = np.triu(np.ones((m, m))) - np.tril(np.ones((m, m)))
alpha = np.ones(m)
batch_size = 10**7 #2.0sec for 10**7 samples

continued = False
if continued:
    with open(DATA_DIR / '5_2_sim_transitive.dat', 'rb') as file:
        packed = file.read()
        batch_results = list(unpack('d' * (len(packed) // 8), packed)) # 8 bytes per double
else:
    batch_results = []

for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
    lotteries = rng.dirichlet(alpha, (3, batch_size))
    comp01 = (np.sum(lotteries[0,:,:] @ ssb_tr * lotteries[1,:,:], axis=1) > 0).astype(int)
    comp12 = (np.sum(lotteries[1,:,:] @ ssb_tr * lotteries[2,:,:], axis=1) > 0).astype(int)
    comp20 = (np.sum(lotteries[2,:,:] @ ssb_tr * lotteries[0,:,:], axis=1) > 0).astype(int)
    num_intr = np.sum(comp01 + comp12 + comp20 == 0) + np.sum(comp01 + comp12 + comp20 == 3)
    batch_results.append(num_intr/batch_size)

with open(DATA_DIR / '5_2_sim_transitive.dat', 'wb') as file:
    file.write(pack('d' * len(batch_results) , *batch_results))

print(np.mean(batch_results), np.std(batch_results), len(batch_results))

0.0013068658490566038 1.1407811612598486e-05 2650


In [3]:
num_samples = 2.65 * 10**10 #1 sample corresponds to 3 lotteries, a triple!
m = 4
rng = np.random.default_rng(2025)
# 3 cycle not preferred to the last option
ssb_intr1 = np.array([[0, 1, 1, 1], [-1, 0, 1, -1], [-1, -1, 0, 1], [-1, 1, -1, 0]])
alpha = np.ones(m)
batch_size = 10**7 #2.5sec for 10**7 samples

continued = False
if continued:
    with open(DATA_DIR / '5_2_sim_intr1.dat', 'rb') as file:
        packed = file.read()
        batch_results = list(unpack('d' * (len(packed) // 8), packed)) # 8 bytes per double
else:
    batch_results = []

for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
    lotteries = rng.dirichlet(alpha, (3, batch_size))
    comp01 = (np.sum(lotteries[0,:,:] @ ssb_intr1 * lotteries[1,:,:], axis=1) > 0).astype(int)
    comp12 = (np.sum(lotteries[1,:,:] @ ssb_intr1 * lotteries[2,:,:], axis=1) > 0).astype(int)
    comp20 = (np.sum(lotteries[2,:,:] @ ssb_intr1 * lotteries[0,:,:], axis=1) > 0).astype(int)
    num_intr = np.sum(comp01 + comp12 + comp20 == 0) + np.sum(comp01 + comp12 + comp20 == 3)
    batch_results.append(num_intr/batch_size)

with open(DATA_DIR / '5_2_sim_intr1.dat', 'wb') as file:
    file.write(pack('d' * len(batch_results) , *batch_results))

print(np.mean(batch_results), np.std(batch_results), len(batch_results))

0.04799177486792453 6.815374696216313e-05 2650


In [4]:
num_samples = 2.65 * 10**10 #1 sample corresponds to 3 lotteries, a triple!
m = 4
rng = np.random.default_rng(2025)
# 4 cycle
ssb_intr2 = np.array([[0, 1, 1, -1], [-1, 0, 1, 1], [-1, -1, 0, 1], [1, -1, -1, 0]])
alpha = np.ones(m)
batch_size = 10**7 #2.5sec for 10**7 samples

continued = False
if continued:
    with open(DATA_DIR / '5_2_sim_intr2.dat', 'rb') as file:
        packed = file.read()
        batch_results = list(unpack('d' * (len(packed) // 8), packed)) # 8 bytes per double
else:
    batch_results = []

for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
    lotteries = rng.dirichlet(alpha, (3, batch_size))
    comp01 = (np.sum(lotteries[0,:,:] @ ssb_intr2 * lotteries[1,:,:], axis=1) > 0).astype(int)
    comp12 = (np.sum(lotteries[1,:,:] @ ssb_intr2 * lotteries[2,:,:], axis=1) > 0).astype(int)
    comp20 = (np.sum(lotteries[2,:,:] @ ssb_intr2 * lotteries[0,:,:], axis=1) > 0).astype(int)
    num_intr = np.sum(comp01 + comp12 + comp20 == 0) + np.sum(comp01 + comp12 + comp20 == 3)
    batch_results.append(num_intr/batch_size)

with open(DATA_DIR / '5_2_sim_intr2.dat', 'wb') as file:
    file.write(pack('d' * len(batch_results) , *batch_results))

print(np.mean(batch_results), np.std(batch_results), len(batch_results))

0.12136223762264152 0.00010404437409472577 2650
